In [1]:
!pip install happytransformer


  Using cached happytransformer-2.4.1-py3-none-any.whl (45 kB)
     |████████████████████████████████| 133.6 MB 655 kB/s eta 0:00:01     |████████████                    | 49.7 MB 14.3 MB/s eta 0:00:06�█▊  | 124.0 MB 770 kB/s eta 0:00:13
  Using cached transformers-4.20.1-py3-none-any.whl (4.4 MB)
     |████████████████████████████████| 365 kB 37.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 44.1 MB/s eta 0:00:01
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     |████████████████████████████████| 132 kB 74.8 MB/s eta 0:00:01
     |████████████████████████████████| 22.4 MB 662 kB/s eta 0:00:011
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached huggingface_hub-0.8.1-py3-none-any.whl (101 kB)
     |████████████████████████████████| 3.6 MB 78.9 MB/s eta 0:00:01


In [1]:
import requests
import json

import re

from happytransformer import HappyGeneration
from happytransformer import GENSettings

/Users/kimkwangil/opt/anaconda3/envs/py38NLP/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

happy_gen = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-1.3B")

def GenText(input_txt):
    tmp = [0.01]

    for temp in tmp:
        top_k_sampling_settings = GENSettings(do_sample=True, 
                                                top_k=120, 
                                                temperature=temp,  
                                                max_length=10, 
                                                no_repeat_ngram_size=2)

        result_top_k_sampling = happy_gen.generate_text(input_txt, args=top_k_sampling_settings)

        #전처리 필요함 GenerationResult(text=' 
        result_top_k_sampling_re = result_top_k_sampling.text
        result_top_k_sampling_re.strip("'")
        result_top_k_sampling_re.lstrip("GenerationResult(text=' ")
        result_top_k_sampling_re = re.sub(r"\n", "", result_top_k_sampling_re)

    return result_top_k_sampling_re



def startChat():
    input_text = input("You- ")
    response = GenText(input_text)
    print("AI :", response)
    return response


#initiate the conversation
if __name__ == "__main__":
    while True:
        startChat()

07/26/2022 14:35:58 - INFO - happytransformer.happy_transformer -   Using model: cpu


You- You poured yourself a glass of cranberry juice, but then absentmindedly, you poured about a teaspoon of bleach into it. It looks OK. You try sniffing it, but you have a bad cold, so you can’t smell anything. You are very thirsty. So you drink it.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI : You are not a very good cook. But you are a good baker. And you make a cake. The cake is delicious. Your mother is very proud of you. She is proud that you made a beautiful cake for her. That is
You- You poured yourself a glass of cranberry juice, but then absentmindedly, you poured about a teaspoon of bleach into it. It looks OK. You try sniffing it, but you have a bad cold, so you can’t smell anything. You are very thirsty. So you drink it.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI : You are not a very good cook. But you are a good baker. And you make a cake. The cake is delicious. Your mother is very proud of you. She is proud that you made a beautiful cake for her. That is


In [ ]:
# 사용자가 입력한 완성된 글(논리적이거나 아니거나)
# AI 는 사용자가 입력한 문장들을 개별 문장으로 나누고, 각 문장을 입력으로 새로운 문장을 자체 생성한다. 생성한 문장은 논리적인 문장일 것이라는 가설.(테스트 완료)
# 사용자가 입력한 문장들을 개별 분장으로 분해해서 리스트에 담는다. 
# 문서유사도를 비교하는 함수를 작동시킨다. (AI가 생성한 문장 1개  VS. 인간입력문장중 1개)
# 문서유사도 결과는 확률값으로 나온다. 예) 0.7 --> 70% 유사하다. 
# 정교하게 만들기 위해서 AI 10개를 돌리고, 합산 후 비교평균값을 산출하여 개별문장 점수를 얻어내고
# 모든 인간 입력 문장에 대한 비교점수를 합산 후 평균을 내면, 결과점수는 바로 논리적인지 아닌지 알 수 있는 수치로 출력된다. 

# 문장 유사도 비교 분석 방법 1

In [3]:
input_sent_A = """Three years later, the coffin was still full of Jello."""
input_sent_B = """The person box was packed with jelly many dozens of months later."""
sen = [input_sent_A, input_sent_B]

In [4]:

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
#Encoding:
sen_embeddings = model.encode(sen)
sen_embeddings.shape


(2, 768)

In [5]:
sen_embeddings

array([[ 0.07446161,  0.8636967 ,  0.1794639 , ...,  0.7734398 ,
         1.7247485 , -0.180275  ],
       [-0.502982  ,  0.79498565, -0.12402511, ...,  0.14406323,
         0.970375  , -0.17911582]], dtype=float32)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
#let's calculate cosine similarity for sentence 0:
sim_re = cosine_similarity(
    [sen_embeddings[0]],
    sen_embeddings[1:]
)

# 문장 유사도 비교 결과 input_sent_A. vs. input_sent_B
sim_re[0][0]

0.72192574

# 문장 유사도 비교 분석 방법 2

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
#nitialize our model and tokenizer:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

###Tokenize the sentences like before:
input_sent_A = """Three years later, the coffin was still full of Jello."""
input_sent_B = """The person box was packed with jelly many dozens of months later."""

sent = [input_sent_A, input_sent_B]

# initialize dictionary: stores tokenized sentences
token = {'input_ids': [], 'attention_mask': []}
for sentence in sent:
    # encode each sentence, append to dictionary
    new_token = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    token['input_ids'].append(new_token['input_ids'][0])
    token['attention_mask'].append(new_token['attention_mask'][0])
# reformat list of tensors to single tensor
token['input_ids'] = torch.stack(token['input_ids'])
token['attention_mask'] = torch.stack(token['attention_mask'])

In [11]:
#Process tokens through model:
output = model(**token)
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [12]:
#The dense vector representations of text are contained within the outputs 'last_hidden_state' tensor
embeddings = output.last_hidden_state
embeddings

tensor([[[-0.0692,  0.6230,  0.0354,  ...,  0.8033,  1.6314,  0.3281],
         [ 0.0367,  0.6842,  0.1946,  ...,  0.0848,  1.4747, -0.3008],
         [-0.0121,  0.6543, -0.0727,  ..., -0.0326,  1.7717, -0.6812],
         ...,
         [ 0.1953,  1.1085,  0.3390,  ...,  1.2826,  1.0114, -0.0728],
         [ 0.0902,  1.0288,  0.3297,  ...,  1.2940,  0.9865, -0.1113],
         [ 0.1240,  0.9737,  0.3933,  ...,  1.1359,  0.8768, -0.1043]],

        [[-0.7576,  0.8399, -0.3792,  ...,  0.1271,  1.2514,  0.1365],
         [-0.6591,  0.7613, -0.4662,  ...,  0.2259,  1.1289, -0.3611],
         [-0.9007,  0.6791, -0.3778,  ...,  0.1142,  0.9080, -0.1830],
         ...,
         [-0.2158,  0.5463,  0.3117,  ...,  0.1802,  0.7169, -0.0672],
         [-0.3092,  0.4833,  0.3021,  ...,  0.2289,  0.6656, -0.0932],
         [-0.2940,  0.4678,  0.3095,  ...,  0.2782,  0.5144, -0.1021]]],
       grad_fn=<NativeLayerNormBackward0>)

In [13]:
embeddings.shape

torch.Size([2, 128, 768])

In [14]:
# To perform this operation, we first resize our attention_mask tensor:
att_mask = token['attention_mask']
att_mask.shape

torch.Size([2, 128])

In [15]:
mask = att_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([2, 128, 768])

In [16]:
mask_embeddings = embeddings * mask
mask_embeddings.shape

torch.Size([2, 128, 768])

In [17]:
#Then we sum the remained of the embeddings along axis 1:
summed = torch.sum(mask_embeddings, 1)
summed.shape

torch.Size([2, 768])

In [18]:
#Then sum the number of values that must be given attention in each position of the tensor:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([2, 768])

In [19]:
mean_pooled = summed / summed_mask
mean_pooled


tensor([[ 0.0745,  0.8637,  0.1795,  ...,  0.7734,  1.7247, -0.1803],
        [-0.5030,  0.7950, -0.1240,  ...,  0.1441,  0.9704, -0.1791]],
       grad_fn=<DivBackward0>)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
#Let's calculate cosine similarity for sentence 0:
# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()
# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.72192585]], dtype=float32)

In [21]:
sim_re = cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

sim_re

array([[0.72192585]], dtype=float32)

In [22]:
type(sim_re)

numpy.ndarray

In [23]:
# 문장 유사도 비교 분석 결과  --- 잘됨. 그러나 코드가 길다. 
sim_re[0][0]

0.72192585